#### Стандартная версия
Нужно реализовать rest api на базе flask в google colab.

выбрать себе датасет (который интересен или нравится больше всего, можно глянуть здесь https://economic-caper-a4c.notion.site/d062c410b90145bca90fc23b1348c813), сделать pipeline (преобразования + модель), сохранить его на диск. Если не хочется пайплайн, то можно без него, но так вам же будет удобнее потом вызывать его из кода сервиса.
Реализовать ноутбук с сервером
Реализовать ноутбук с клиентом

# Интеграция.Итоговый проект

In [1]:
import pandas as pd
import dill
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
from sklearn.metrics import roc_auc_score, roc_curve, precision_recall_curve
from sklearn.metrics import f1_score

#working with text
from sklearn.feature_extraction.text import TfidfVectorizer

#normalizing data
from sklearn.preprocessing import StandardScaler

#pipeline
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.metrics import precision_score,recall_score

#imputer
from sklearn.impute import SimpleImputer

import sklearn.datasets

In [2]:
df = pd.read_csv("train.csv")
df.head(3)
df.tail()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
159566,ffe987279560d7ff,""":::::And for the second time of asking, when ...",0,0,0,0,0,0
159567,ffea4adeee384e90,You should be ashamed of yourself \n\nThat is ...,0,0,0,0,0,0
159568,ffee36eab5c267c9,"Spitzer \n\nUmm, theres no actual article for ...",0,0,0,0,0,0
159569,fff125370e4aaaf3,And it looks like it was actually you who put ...,0,0,0,0,0,0
159570,fff46fc426af1f9a,"""\nAnd ... I really don't think you understand...",0,0,0,0,0,0


##### Я так понял, что датасет на выявление токсичности в тексте.Задача бинарной классификации .Целевая переменная---identity_hate

In [3]:
df['identity_hate'].value_counts()

0    158166
1      1405
Name: identity_hate, dtype: int64

##### Наблюдается дисбаланс класса

#### Разделим данные на train/test и сохраним тестовую выборку на диск!!!(X_test.csv, y_test.csv)

In [4]:
X_train, X_test, y_train, y_test = train_test_split(df, df['identity_hate'],
                                                    test_size=0.33, random_state=42)# разбиение 1:3
# save test
X_test.to_csv("X_test.csv", index=None)
y_test.to_csv("y_test.csv", index=None)

# save train
X_train.to_csv("X_train.csv", index=None)
y_train.to_csv("y_train.csv", index=None)

##### Напишем классы для пайплайна

In [5]:
class ColumnSelector(BaseEstimator, TransformerMixin): # Выбор одного признака
    """
    Transformer to select a single column from the data frame to perform additional transformations on
    """
    def __init__(self, key):
        self.key = key

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return X[self.key]
    

class TextImputer(BaseEstimator, TransformerMixin): # Класс заполнения пропусков в текстовом признаке 'comment_text'
    def __init__(self, key, value):
        self.key = key
        self.value = value
    
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        X[self.key] = X[self.key].fillna(self.value) # меняем пропуски на значение
        return X

In [6]:
features = 'comment_text'
target = 'identity_hate'

##### Соберем кусок, ответственный за feature engineering

In [8]:
# combine
comment_text = Pipeline([
                ('imputer', TextImputer('comment_text', '')),         # меняем незаполненный признак на пустую строку ''
                ('selector', ColumnSelector(key='comment_text')),
                ('tfidf', TfidfVectorizer())              # прогоняем признак через Tfidf векторайзер
            ])




feats = FeatureUnion([('comment_text', comment_text)])
                      
                    

##### Добавим простейший классификатор

In [9]:
%%time

pipeline = Pipeline([
    ('features', feats),
    ('classifier', LogisticRegression()), #  в качестве классификатора берем логистическую регрессию,Tfidf позволяет нам это сделать
])

pipeline.fit(X_train, y_train)

<ipython-input-5-ea939544cf2a>:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X[self.key] = X[self.key].fillna(self.value) # меняем пропуски на значение


Wall time: 17.1 s


Pipeline(steps=[('features',
                 FeatureUnion(transformer_list=[('comment_text',
                                                 Pipeline(steps=[('imputer',
                                                                  TextImputer(key='comment_text',
                                                                              value='')),
                                                                 ('selector',
                                                                  ColumnSelector(key='comment_text')),
                                                                 ('tfidf',
                                                                  TfidfVectorizer())]))])),
                ('classifier', LogisticRegression())])

#### Сохраним модель (пайплайн).Один признак и одна модель

In [10]:
with open("logreg_pipeline.dill", "wb") as f:  
    dill.dump(pipeline, f)

##### Сохранем модель в формате dill весь наш пайплайн,это избавляет нас от лишних ошибок в будущем

### Шаг 2.Предсказание (Predict)

##### Проверка работоспособности и качества пайплайна

##### Загружаем модель напрямую и проверяем на отложенной(тестовой ) выборке

In [11]:
X_test = pd.read_csv("X_test.csv")
y_test = pd.read_csv("y_test.csv")

In [12]:
X_test.head(3)

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,7ca72b5b9c688e9e,"Geez, are you forgetful! We've already discus...",0,0,0,0,0,0
1,c03f72fd8f8bf54f,Carioca RFA \n\nThanks for your support on my ...,0,0,0,0,0,0
2,9e5b8e8fc1ff2e84,"""\n\n Birthday \n\nNo worries, It's what I do ...",0,0,0,0,0,0


##### Теперь загрузим модель

In [13]:
with open('logreg_pipeline.dill', 'rb') as in_strm:
    pipeline = dill.load(in_strm)

In [14]:
pipeline

Pipeline(steps=[('features',
                 FeatureUnion(transformer_list=[('comment_text',
                                                 Pipeline(steps=[('imputer',
                                                                  TextImputer(key='comment_text',
                                                                              value='')),
                                                                 ('selector',
                                                                  ColumnSelector(key='comment_text')),
                                                                 ('tfidf',
                                                                  TfidfVectorizer())]))])),
                ('classifier', LogisticRegression())])

##### Здесь наши признаки,где мы заполняем пустоты при помощи Tfidf векторайзера

In [15]:
preds = pipeline.predict_proba(X_test)[:, 1]  # предсказание вероятности,чтобы подообрать необходимую отсечку

pred_df = pd.DataFrame({'preds': preds})
pred_df.to_csv("test_predictions.csv", index=None) # сохраняем предсказание в файле,хорошая стратегия для логирования при обучении многих моделей

In [16]:
preds[:10]

array([0.01481612, 0.00149461, 0.00782364, 0.00187916, 0.00287763,
       0.00197865, 0.00567359, 0.01054992, 0.000868  , 0.00685097])

In [17]:
precision, recall, thresholds = precision_recall_curve(y_test, preds) # подбираем оптимальную отсечку 'thresholds'

fscore = (2 * precision * recall) / (precision + recall)
# locate the index of the largest f score
ix = np.argmax(fscore)
print(f'Best Threshold={thresholds[ix]}, F-Score={fscore[ix]:.3f}, Precision={precision[ix]:.3f}, Recall={recall[ix]:.3f}')

Best Threshold=0.15290741374698139, F-Score=0.442, Precision=0.490, Recall=0.403


##### Метрики получились не очень хорошие,вероятно из-за того,что использовали один признак

### Шаг 3.Flask

##### Flask

Тут будет сервис для обработки запросов на Flask

In [18]:

from flask import Flask, request, jsonify
import pandas as pd

In [ ]:
# Пробный запуск Flask

app = Flask(__name__)


@app.route("/a")
def hello():
    return "Hello World!"

if __name__ == '__main__':  # попросили меня пользоваться другим сервером,каким---не знаю.Машина встала
    app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [27/Apr/2022 21:55:03] "GET / HTTP/1.1" 404 -
127.0.0.1 - - [27/Apr/2022 21:55:03] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [27/Apr/2022 21:55:18] "GET / HTTP/1.1" 404 -


In [ ]:
import pandas as pd
import dill

### Создаем сервис для обработки запросов к модели

In [ ]:
# Загружаем обученные модели
with open('logreg_pipeline.dill', 'rb') as in_strm:
    model = dill.load(in_strm)

In [ ]:
X_test = pd.read_csv("X_test.csv")
y_test = pd.read_csv("y_test.csv")

##### Запускаем сервис

In [ ]:
# Обработчики и запуск Flask
app = Flask(__name__)



@app.route("/", methods=["GET"])  # главный путь
def general():
    return "Welcome to prediction process"


@app.route('/predict', methods=['POST'])  # метод PREDICT,ждём JSON
def predict():
    data = {"success": False}  # формируем словарь в JSON и отправляем клиенту.False по дефолту

    # ensure an image was properly uploaded to our endpoint
    comment_text = "", "", ""
    request_json = request.get_json()
    
    if request_json["comment_text"]:
        comment_text = request_json['comment_text']
    
    
    
    print(comment_text)  
    preds = model.predict_proba(pd.DataFrame({"comment_text": [comment_text]
                                              
                                              }))   # формируем один объект для предсказания вероятности по датафрейму
    data["predictions"] = preds[:, 1][0] # 1-целевой класс,0-это наш объект.В данном случае он один--comment_text
    data["comment_text"] = comment_text
        # indicate that the request was a success
    data["success"] = True  # модель предсказала
    print('OK')

        # return the data dictionary as a JSON response
    return jsonify(data)    # перевод словаря в JSON


if __name__ == '__main__':    # запуск приложения
    app.run()

### Строим запрос к нашему сервису

In [ ]:
import requests

import urllib.request
import json 

In [ ]:
# Пример данных
data = ( 
    "Stylect is a dynamic startup that helps helps women discover and buy shoes. We’re a small team based in London that has previously worked at Google, Techstars, Pixelmator and Rocket Internet.We place a high premium on simplicity no matter what we’re working on (i.e. design, programming, marketing). We’re also a team that ships fast. We built version 1 of our app in a week, the next release (built in a month) was featured in the Apple Appstore Italy as a best new fashion app. Fast release cycles are challenging, but also very fun - which is why we love them.\xa0As we’ve grown, the projects that we’re working on have grown both in scale and in technical complexity. \xa0Stylect is looking for someone who can help us improve our backend which gathers product data; analyses/categorizes it; and shows it to thousands of users daily. Each step in the process has unique challenges that demands a strong technical background.",
    "ustwo offers you the opportunity to be yourself, whilst delivering the best work on the planet for some of the biggest and most innovative brands. A culture thriving on collaboration underpins what is an amazing work smart/ live well environment.We genuinely care about the work that we deliver and the people who help make it all possible. We only invest in projects, people and practices that we believe in, to ensure we remain excited about every opportunity.",
    "We are negotiable on salary and there is the potential for equity for the right candidate."
)

In [ ]:
# формируем запрос
def send_json(x):
    comment_text = x
    #print(comment_text)  # наш признак
    body = {
        'comment_text': comment_text    # тело запроса
        
        }
    myurl = '/predict'
    headers = {'content-type': 'application/json; charset=utf-8'}  # шапка
    response = requests.post(myurl, json=body, headers=headers) # запрос по нашему пути(Post),телу запроса,шапкой
    return response.json()['predictions']

In [ ]:
# обращение к серверу с запросом из одного набора (его построили руками выше - data)
response = send_json(data)
print('предсказание', response)

##### Здесь должно быть предсказание.Но из-за того,что компьютер встал,результат не виден

### Сделаем обработку массы запросов:

1. загрузим данные X_test



In [ ]:
import pandas as pd
from sklearn.metrics import roc_auc_score, roc_curve, precision_recall_curve
from sklearn.metrics import f1_score
from urllib import request, parse


X_test = pd.read_csv("X_test.csv")
y_test = pd.read_csv("y_test.csv")

##### 2. Передаем запрос одиночный из наших данных и посмотрим на ответ

In [ ]:
response = send_json(X_test[['comment_text']].iloc[0,:])
response   # должна быть какая-то вероятность

In [ ]:
X_test[['comment_text']].head()

##### 3. Сделаем N запросов и оценим время

In [ ]:
N = 50

In [ ]:
%%time
predictions = X_test[
                     ['comment_text']
                     ].iloc[:N].apply(lambda x: send_json(x), axis=1) # здесь заданы 50 объектов,каждый объект отправим на предсказание

In [ ]:
predictions.values[:5]

##### 4. Посчитаем метрику

In [ ]:
y_test.head() # посчитаем метрику identity_hate

In [ ]:
import numpy as np

precision, recall, thresholds = precision_recall_curve(y_test[:N], predictions)

fscore = (2 * precision * recall) / (precision + recall)
# locate the index of the largest f score
ix = np.argmax(fscore)
print(f'Best Threshold={thresholds[ix]}, F-Score={fscore[ix]:.3f}, Precision={precision[ix]:.3f}, Recall={recall[ix]:.3f}')

In [ ]:
Итог: Flask слишком тяжёлый для компа,он завис и результаа я не увидел.

##### Тестовый клиент

In [ ]:
# Пример данных.Создание тестового объекта и формирование тела запроса
comment_text_data = ( 
    "Stylect is a dynamic startup that helps helps women discover and buy shoes. We’re a small team based in London that has previously worked at Google, Techstars, Pixelmator and Rocket Internet.We place a high premium on simplicity no matter what we’re working on (i.e. design, programming, marketing). We’re also a team that ships fast. We built version 1 of our app in a week, the next release (built in a month) was featured in the Apple Appstore Italy as a best new fashion app. Fast release cycles are challenging, but also very fun - which is why we love them.\xa0As we’ve grown, the projects that we’re working on have grown both in scale and in technical complexity. \xa0Stylect is looking for someone who can help us improve our backend which gathers product data; analyses/categorizes it; and shows it to thousands of users daily. Each step in the process has unique challenges that demands a strong technical background.",
    "ustwo offers you the opportunity to be yourself, whilst delivering the best work on the planet for some of the biggest and most innovative brands. A culture thriving on collaboration underpins what is an amazing work smart/ live well environment.We genuinely care about the work that we deliver and the people who help make it all possible. We only invest in projects, people and practices that we believe in, to ensure we remain excited about every opportunity.",
    "We are negotiable on salary and there is the potential for equity for the right candidate."
)

body = {
        'comment_text': comment_text 
        
        
        }

In [ ]:
with app.test_client() as t:
    response = t.post('/predict', json=body)
    json_data = response.get_json()

json_data